### Install libs

In [ ]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install -q lightning-bolts

### Import libs

In [18]:
# Sistema e manipulação de arquivos
import os

# Manipulação e processamento de dados
import pandas as pd
import numpy as np
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

# Modelagem
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn

# Avaliação do modelo
from torchmetrics.classification import Accuracy

# Visualização de dados
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns


### Configurando a GPU para processamento

In [13]:
processing_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_aval = torch.cuda.is_available()

# Definindo o método de processamento.
torch.device(processing_device)

device(type='cuda')

In [9]:
# Limpando a memória da GPU
torch.cuda.empty_cache()

In [10]:
# Informações da GPU
!nvidia-smi

Tue Apr  9 13:30:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.12                 Driver Version: 552.12         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080      WDDM  |   00000000:0B:00.0  On |                  N/A |
|  0%   39C    P8             18W /   90W |    1033MiB /   8192MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

device(type='cuda')